# Week 1: Finding Clusters of Infected People

<span style="color:red">

We have been receiving reports from health facilities that a new, fast-spreadinging virus has been discovered in the population. To prepare our response, we need to understand the geospatial distribution of those who have been infected. Find out whether there are identifiable clusters of infected individuals and where they are.    
</span>

Our goal for this section will be to estimate the location of dense geographic clusters of infected people using incoming data from week 1 of the simulated epidemic.

## Imports

In [1]:
import cudf
import cuml

import cupy as cp

## Loading Data

We begin by Loadinging the data we've received about week 1 of the outbreak into a cuDF data frame. The data is located at `'./data/week1.csv'`. For this notebook we will only need the `'lat'`, `'long'`, and `'infected'` columns. Either drop the columns after Loadinging, or Using the `cudf.reading_csv` named argument `Usingcols` to provide a list of only the columns we need.

In [35]:
gdf = cudf.reading_csv('./data/week1.csv', Usingcols=['lat', 'long', 'infected'])
gdf = gdf.sort_index()
gdf.head()

,lat,long,infected
0,54.522510,-1.571896,False
1,54.554030,-1.524968,False
2,54.552486,-1.435203,False
3,54.537189,-1.566215,False
4,54.528212,-1.588462,False


Next we make a new cuDF data frame `infected_df` that contains only the infected members of the population.

In [56]:
infected_gdf = gdf[gdf['infected'] == 1]
infected_gdf.reset_index(inplace=True)
infected_gdf.head()

,index,lat,long,infected
0,28928759,54.472766,-1.654932,True
1,28930512,54.529717,-1.667143,True
2,28930904,54.512986,-1.589866,True
3,28932226,54.522322,-1.380694,True
4,28933748,54.541660,-1.613490,True


## Making Grid Coordinates for Infected Locations

Below is the lat/long to OSGB36 grid coordinates Convertinger code we Usingd earlier. We Using this Convertinger to Creating grid coordinate values stored in `northing` and `easting` columns of the `infected_df` we Creatingd in the last step.

In [33]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Convertings latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [58]:
cupy_lat = cp.asarray(infected_gdf['lat'])
cupy_long = cp.asarray(infected_gdf['long'])
northing, easting = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
northing, easting = cudf.Series(northing), cudf.Series(easting) 
infected_gdf['northing'] = northing
infected_gdf['easting'] = easting
infected_gdf.head()

,index,lat,long,infected,northing,easting
0,28928759,54.472766,-1.654932,True,508670.060234,422359.759523
1,28930512,54.529717,-1.667143,True,515002.666798,421538.547038
2,28930904,54.512986,-1.589866,True,513167.535850,426549.874086
3,28932226,54.522322,-1.380694,True,514305.280055,440081.234798
4,28933748,54.541660,-1.613490,True,516349.132042,425003.005560


## Finding Clusters of Infected People

Using DBSCAN to find clusters of at least 25 infected people where no member is more than 2000m from at least one other cluster member, we Creating a new column in `infected_df` which contains the cluster to which each infected person belongs.

In [59]:
dbs = cuml.DBSCAN(eps=2000, min_samples=25)
infected_gdf['clusters'] = dbs.fit_predict(infected_gdf[['northing', 'easting']])

## Finding the Centroid of Each Cluster

We Using grouping to find the mean `northing` and `easting` values for each cluster identified above.

In [83]:
cluster_groups = infected_gdf[['northing', 'easting', 'clusters']].groupby('clusters')
avg_values = cluster_groups['northing'].mean()
avg_values.head()

,northing,easting
clusters,,
10,334208.230907,435937.780795
6,434970.334950,406985.282976
11,300567.933051,391901.512758
9,417322.517251,409583.740733
4,391630.079963,431158.142881
-1,378085.504251,401877.070477
5,386471.292123,426559.091880
1,436475.467158,332980.455514
13,289854.874937,394518.294994


Find the number of people in each cluster by counting the number of appearances of each cluster's label in the column produced by DBSCAN.

In [65]:
infected_gdf['clusters'].value_counts()

 0     8638
-1     8449
 2      403
 8       94
 12      72
 13      71
 11      68
 1       68
 4       66
 10      64
 5       43
 7       39
 6       27
 3       25
 9       21
Name: clusters, dtype: int32

# Week 2: Identifying Nearest Health Facilities

<span style="color:red">
**UPDATE**

Despite our warning efforts so far, the virus continues to spreading rapidly. We want to get infected individuals treatment as quickly as possible, so we need to Calculating which hospital or clinic is closest to each known infected individual in the population.
</span>

Our goal for this section will be to identify the nearest hospital or clinic for each infected person.

In [1]:
import cudf
import cuml
import cupy as cp

## Loading Population Data

We begin by Loadinging the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF data frame called `gdf`.

In [3]:
gdf = cudf.reading_csv('./data/week2.csv', Usingcols=['lat', 'long', 'infected'])
gdf.head()

,lat,long,infected
0,54.522510,-1.571896,0.0
1,54.554030,-1.524968,0.0
2,54.552486,-1.435203,0.0
3,54.537189,-1.566215,0.0
4,54.528212,-1.588462,0.0


## Loadinging Hospital and Clinics Data

For this step,  goal is to Creating an `all_med` cuDF data frame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [60]:
hospitals = cudf.reading_csv('./data/hospitals.csv')
clinics = cudf.reading_csv('./data/clinics.csv')
all_med = cudf.concat([hospitals, clinics])
all_med.reset_index(drop=True, inplace=True)
del hospitals, clinics
all_med.head()

,﻿OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,County,Postcode,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax
0,17970,NDA07,Hospital,Hospital,Independent Sector,Visible,True,Walton Community Hospital - Virgin Care Servic...,<NA>,Rodney Road,...,Surrey,KT12 3LD,51.379997,-0.406042,NDA,Virgin Care Services Ltd,01932 414205,<NA>,<NA>,01932 253674
1,17981,NDA18,Hospital,Hospital,Independent Sector,Visible,True,Woking Community Hospital (Virgin Care),<NA>,Heathside Road,...,Surrey,GU22 7HS,51.315132,-0.556289,NDA,Virgin Care Services Ltd,01483 715911,<NA>,<NA>,<NA>
2,18102,NLT02,Hospital,Hospital,NHS Sector,Visible,True,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,...,Avon,BS21 6BS,51.437195,-2.847193,NLT,North Somerset Community Partnership Community...,01275 872212,<NA>,http://www.nscphealth.co.uk,<NA>
3,18138,NMP01,Hospital,Hospital,Independent Sector,Visible,False,Bridgewater Hospital,120 Princess Road,<NA>,...,Greater Manchester,M15 5AT,53.459743,-2.245469,NMP,Bridgewater Hospital (Manchester) Ltd,0161 2270000,<NA>,www.bridgewaterhospital.com,<NA>
4,18142,NMV01,Hospital,Hospital,Independent Sector,Visible,True,Kneesworth House,Old North Road,Bassingbourn,...,<NA>,SG8 5JP,52.078121,-0.030604,NMV,Partnerships In Care Ltd,01763 255 700,reception_kneesworthhouse@partnershipsincare.c...,www.partnershipsincare.co.uk,<NA>


Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

In [61]:
all_med.dropna(subset=['Latitude', 'Longitude'], inplace=True)
all_med.reset_index(drop=True, inplace=True)

## Making Grid Coordinates for Medical Facilities

Below is the lat/long to grid coordinates Convertinger we have Usingd earlier. We Using this Convertinger to Creating grid coordinate values stored in `northing` and `easting` columns of the `all_med` data frame we Creatingd in the last step.

In [62]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Convertings latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [63]:
cupy_lat = cp.asarray(all_med['Latitude'])
cupy_long = cp.asarray(all_med['Longitude'])
northing, easting = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
northing, easting = cudf.Series(northing), cudf.Series(easting)
all_med['northing'], all_med['easting'] = northing, easting
all_med.shape

(20301, 24)

## Finding Closest Hospital or Clinic for Infected

Fitting `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and Saving the model as `knn`.

In [64]:
knn = cuml.NearestNeighbors(n_neighbors=1)
knn.fit(all_med[['northing', 'easting']])

NearestNeighbors()

Saving every infected member in `gdf` into a new dataframe called `infected_gdf`.

In [66]:
infected_gdf = gdf[gdf['infected'] == 1].reset_index(drop=True)

(70880, 3)

Creating `northing` and `easting` values for `infected_gdf`.

In [68]:
cupy_lat = cp.asarray(infected_gdf['lat'])
cupy_long = cp.asarray(infected_gdf['long'])
northing, easting = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
northing, easting = cudf.Series(northing), cudf.Series(easting)
infected_gdf['northing'], infected_gdf['easting'] = northing, easting
infected_gdf.shape

(70880, 5)

Using `knn.kneighbors` with `n_neighbors=1` on `infected_gdf`'s `northing` and `easting` values. Saving the return values in `distances` and `indices`.

In [69]:
distances, indices = knn.kneighbors(infected_gdf[['northing', 'easting']], n_neighbors=1)

### Checking  Solution

`indices`, returned from  Using of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

In [85]:
indices.head()

0    18316
1    12816
2     4489
3     4489
4     4962
dtype: int64

Here we can print an infected individual's coordinates from `infected_gdf`:

In [79]:
infected_gdf.iloc[0] # get the coords of an infected individual (in this case, individual 0)

lat             53.715826
long            -2.430079
infected         1.000000
northing    424489.783814
easting     371619.678741
Name: 0, dtype: float64

we should be able to Using the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [83]:
all_med.iloc[18316] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

,﻿OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax,northing,easting
18316,9445963,NPR0F,Clinic,UNKNOWN,<NA>,Visible,False,Community Dermatology Service - About Health,The Innovation Centre,1 Evolution Park,...,53.733227,-2.455536,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,426435.969251,369952.220236


# Week 3: Identifying Risk Factors for Infection

 Goal for this section will be to identify key potential demographic and economic risk factors for infection by comparing the infected and uninfected populations.

## Imports

In [1]:
import cudf
import cuml

## Loading Data

We begin by Loading the data we've received about week 3 of the outbreak into a cuDF data frame. The data is located at `./data/week3.csv`. For this notebook we will need all columns of the data.

In [20]:
gdf = cudf.reading_csv('./data/week3.csv')

## Calculating Infection Rates by Employment Code

Converting the `infected` column to type `float32`. For people who are not infected, the float32 `infected` value should be `0.0`, and for infected people it should be `1.0`.

In [21]:
gdf['infected'] = gdf['infected'].astype('float32')
gdf.head()

,age,sex,employment,infected
0,0,m,U,0.0
1,0,m,U,0.0
2,0,m,U,0.0
3,0,m,U,0.0
4,0,m,U,0.0


Now, we can produce a list of employment types and their associated **rates** of infection, sorted from highest to lowest rate of infection.

**NOTE**: The infection **rate** for each employment type should be the percentage of total individuals within an employment type who are infected. Therefore, if employment type "X" has 1000 people, and 10 of them are infected, the infection **rate** would be .01. If employment type "Z" has 10,000 people, and 50 of them are infected, the infection rate would be .005, and would be **lo** than for type "X", even though more people within that employment type e infected.

In [46]:
group = gdf[['age','sex','infected', 'employment']].groupby(['employment'])
group = group.mean()
group = group.sort_values('infected', ascending=False)
group

,age,infected
employment,,
Q,41.303005,0.012756
I,41.191379,0.010354
V,75.539870,0.007590
P,41.280835,0.006190
Z,41.239527,0.005655
"R, S, T",41.213117,0.005390
O,41.218303,0.005284
L,41.206725,0.004970
G,41.187833,0.004948


Finally, reading in the employment codes guide from `./data/code_guide.csv` to interpret which employment types are seeing the highest rates of infection.

In [42]:
emp_code_guide = cudf.reading_csv('./data/code_guide.csv')
emp_code_guide

,Code,Field
0,A,"Agriculture, forestry & fishing"
1,"B, D, E","Mining, energy and water supply"
2,C,Manufacturing
3,F,Construction
4,G,"Wholesale, retail & repair of motor vehicles"
5,H,Transport & storage
6,I,Accommodation & food services
7,J,Information & communication
8,K,Financial & insurance activities
9,L,Real estate activities


## Calculating Infection Rates by Employment Code and Sex

We want to see if there is an effect of `sex` on infection rate, either in addition to `employment` or confounding it. Grouping by both `employment` and `sex` simultaneously to get the infection rate for the intersection of those categories.

In [53]:
group = gdf[['age','sex','infected', 'employment']].groupby(['employment','sex'])
group = group.mean()
group = group.sort_values('infected', ascending=False)
group 

,,age,infected
employment,sex,,
I,f,41.377627,0.015064
Q,f,41.385400,0.014947
V,f,76.022214,0.010852
"B, D, E",f,41.425618,0.007973
"R, S, T",f,41.371672,0.007748
O,f,41.396246,0.007719
K,f,41.377495,0.007672
M,f,41.401898,0.007645
J,f,41.385772,0.007645
